### BA5A | The Change Problem

In [1]:
money = 40
coins = [1,5,10,20,25,50]

In [2]:
def DPchange(money, coins):
    mincoins = [float("inf")]*(money+1)
    mincoins[0] = 0
    for m in range(1, money+1):
        for i in range(len(coins)):
            if m >= coins[i]:
                if mincoins[m-coins[i]]+1 < mincoins[m]:
                    mincoins[m] = mincoins[m-coins[i]]+1
    return mincoins[money]
    

In [3]:
DPchange(money, coins)

2

### BA5B | Length of a Longest Path in the Manhattan Tourist Problem

In [4]:
with open('../data/rosalind_ba5b.txt', 'r') as f:
    n, m = [int(i) for i in f.readline().strip().split()]
    lines = f.readlines()
    brk_idx = lines.index('-\n')
    down = [[int(x) for x in l.strip().split()] for l in lines[:brk_idx]]
    right = [[int(x) for x in l.strip().split()] for l in lines[brk_idx+1:]]

In [5]:
def Manhattan(n, m, down, right):
    s = [[0 for r in range(m+1)] for d in range(n+1)] 
    for i in range(1,n+1):
        s[i][0] = s[i-1][0]+down[i-1][0]
    for j in range(1,m+1):
        s[0][j] = s[0][j-1] + right[0][j-1]

    for i in range(1, n+1): 
        for j in range(1, m+1):
            s[i][j] = max(s[i-1][j]+down[i-1][j], s[i][j-1]+right[i][j-1])
    return s[n][m]

In [6]:
Manhattan(n,m,down,right)

34

### BA5C | Longest Common Subsequence Problem

In [7]:
str1 = 'AACCTTGG'
str2 = 'ACACTGTGA'

In [8]:
def LSC(str1, str2):
    n = len(str1)
    m = len(str2)
    s = [[0 for x in range(m+1)] for y in range(n+1)] 
    backtrack = [[0 for x in range(m+1)] for y in range(n+1)] 
    
    for i in range(1,n+1):
        for j in range(1,m+1):
            if str1[i-1] == str2[j-1]:
                s[i][j] = s[i-1][j-1]+1
                backtrack[i][j] = 1
            else:
                s[i][j] = max(s[i-1][j-1], s[i-1][j], s[i][j-1])
                if s[i][j] == s[i-1][j-1]:
                    backtrack[i][j] = 1
                elif s[i][j] == s[i-1][j]:
                    backtrack[i][j] = 2
                elif s[i][j] == s[i][j-1]:
                    backtrack[i][j] = 3
    i,j = n,m
    lcs = ''
    while s[i][j] > 0:
        if backtrack[i][j] == 1:
            if str1[i-1] == str2[j-1]:
                lcs += str1[i-1]
            i = i-1
            j = j-1
        elif backtrack[i][j] == 2:
            i = i-1
        elif backtrack[i][j] == 3:
            j = j-1
        
    return lcs[::-1] 

In [9]:
LSC(str1, str2)

'AACTTG'

### BA5D |  Longest Path

In [18]:
graph = {}
weights = {}
with open('../data/rosalind_ba5d.txt') as f:
    source = f.readline().strip()
    sink = f.readline().strip()
    lines = [line.strip().split('->') for line in f.readlines()]
    for line in lines:
        from_node = line[0]
        to_node = line[1].split(':')[0]
        weights[(from_node, to_node)] = line[1].split(':')[1]
        if from_node not in graph:
            graph[from_node] = [to_node]
        else:
            graph[from_node].append(to_node)

In [24]:
def TopologicalOrder(graph):
    in_degree = {node:0 for node in graph}
    for u in graph:
        for v in graph[u]:
            if v not in in_degree:
                in_degree[v] = 1
            else:
                in_degree[v]+= 1

    order = []
    candidates = []
    for node in in_degree:
        if in_degree[node] == 0:
            candidates.append(node)

    while len(candidates) > 0:
        a = candidates.pop()
        order.append(a)

        if a in graph:
            for b in graph[a]:
                in_degree[b] -= 1
                if in_degree[b] == 0:
                    candidates.append(b)
    return order

In [25]:
def LongestPath(graph):
    rev_graph = {}
    for u in graph:
        for v in graph[u]:
            if v not in rev_graph:
                rev_graph[v] = [u]
            else:
                rev_graph[v].append(u) 

    # get all nodes in graph
    nodes = set(graph.keys())
    for n in graph.keys():
        nodes = nodes.union(set(graph[n]))

    topological = TopologicalOrder(graph)   

    s = {node: -200 for node in nodes}
    backtrack = {node: None for node in topological}
    s[source] = 0
    
    for node in topological[1:]:
        try:
            s[node] = max([int(s[n])+int(weights[(n,node)]) for n in rev_graph[node]])
            for n in rev_graph[node]:
                if s[node] == int(s[n])+int(weights[(n,node)]):
                    backtrack[node] = n
        except KeyError:
            pass
     
    path = [sink]
    curr = sink
    while curr != source:
        path.append(backtrack[curr])
        curr = backtrack[curr]
    print s[sink]
    print '->'.join(path[::-1])
    return 

In [26]:
LongestPath(graph)

9
0->2->3->4


### BA5E | Global Alignment Problem

In [27]:
from Bio.SubsMat.MatrixInfo import blosum62

In [41]:
def GloabalAlignment(str1, str2, penalty, score_mat):
    n = len(str1)
    m = len(str2)
    s = [[0 for x in range(m+1)] for y in range(n+1)]
    backtrack = [[0 for x in range(m+1)] for y in range(n+1)] 
    
    for i in range(n+1):
        s[i][0] = -i*penalty
    for j in range(m+1):
        s[0][j] = -j*penalty
    for i in range(1,n+1):
        for j in range(1, m+1):
            s[i][j] = max(s[i-1][j-1]+score_mat[(str1[i-1], str2[j-1])], s[i-1][j]-penalty, s[i][j-1]-penalty)
            
            if s[i][j] == (s[i-1][j-1]+score_mat[(str1[i-1], str2[j-1])]):
                backtrack[i][j] = 1
            elif s[i][j] == s[i-1][j]-penalty:
                backtrack[i][j] = 2
            elif s[i][j] == s[i][j-1]-penalty:
                backtrack[i][j] = 3
    
    align1, align2 = '', ''
    i, j = n, m
    while (i >0 ) and (j > 0):
        if backtrack[i][j] == 1:
            align1 += str1[i-1]
            align2 += str2[j-1]
            i = i-1
            j = j-1
        elif backtrack[i][j] == 2:
            align1 += str1[i-1]
            align2 += '-'
            i = i-1
        elif backtrack[i][j] == 3:
            align1 += '-'
            align2 += str2[j-1]
            j = j-1
    print s[n][m]
    print align1[::-1]
    print align2[::-1]
    #return s

In [42]:
str1 = 'PLEASANTLY'
str2 = 'MEANLY'

In [43]:
blosum62_update = {}
for pair in blosum62:
    blosum62_update[pair] = blosum62[pair]
    blosum62_update[(pair[1], pair[0])] = blosum62[pair]

In [44]:
GloabalAlignment(str1, str2, 5, blosum62_update)

8
LEASANTLY
ME--AN-LY


### BA5F | Local Alignment Problem

In [45]:
from Bio.SubsMat.MatrixInfo import pam250

In [46]:
str1 = 'PENALTY'
str2 = 'MEANLY'

In [47]:
def LocalAlignment(str1, str2, penalty, score_mat):
    n = len(str1)
    m = len(str2)
    s = [[0 for x in range(m+1)] for y in range(n+1)]
    backtrack = [[0 for x in range(m+1)] for y in range(n+1)] 
    max_loc= (0,0)
    max_score = 0
    
    for i in range(1,n+1):
        for j in range(1, m+1):
            s[i][j] = max(0, s[i-1][j-1]+score_mat[(str1[i-1], str2[j-1])], s[i-1][j]-penalty, s[i][j-1]-penalty)
            
            if s[i][j] > max_score:
                max_score = s[i][j]
                max_loc = (i, j)
                
            if s[i][j] == (s[i-1][j-1]+score_mat[(str1[i-1], str2[j-1])]):
                backtrack[i][j] = 1
            elif s[i][j] == s[i-1][j]-penalty:
                backtrack[i][j] = 2
            elif s[i][j] == s[i][j-1]-penalty:
                backtrack[i][j] = 3

    align1, align2 = '', ''
    i, j = max_loc[0], max_loc[1]
    
    while s[i][j] > 0:
        if backtrack[i][j] == 1:
            align1 += str1[i-1]
            align2 += str2[j-1]
            i = i-1
            j = j-1
        elif backtrack[i][j] == 2:
            align1 += str1[i-1]
            align2 += '-'
            i = i-1
        elif backtrack[i][j] == 3:
            align1 += '-'
            align2 += str2[j-1]
            j = j-1
        else:
            break
    print max_score
    print align1[::-1]
    print align2[::-1]
    return 

In [48]:
pam250_update = {}
for pair in pam250:
    pam250_update[pair] = pam250[pair]
    pam250_update[(pair[1], pair[0])] = pam250[pair]

In [49]:
LocalAlignment(str1, str2, 5, pam250_update)

15
ENALTY
EANL-Y


### BA5G | Edit Distance Problem

In [53]:
def EditDist(str1, str2):
    n = len(str1)
    m = len(str2)
    s = [[0 for x in range(m+1)] for y in range(n+1)] 
    for i in range(n+1):
        s[i][0] = i
    for j in range(m+1):
        s[0][j] = j
    for i in range(1,n+1):
        for j in range(1,m+1):
            if str1[i-1] == str2[j-1]:
                s[i][j] = s[i-1][j-1]
            else:
                s[i][j] = min(s[i-1][j-1], s[i-1][j], s[i][j-1])+1
    return s[n][m]

In [54]:
str1 = 'PLEASANTLY'
str2 = 'MEANLY'

In [55]:
EditDist(str1, str2)

5

### BA5H | Fitting Alignment

In [50]:
def FittingAlignment(str1, str2):
    n = len(str1)
    m = len(str2)
    s = [[0 for x in range(m+1)] for y in range(n+1)]
    backtrack = [[0 for x in range(m+1)] for y in range(n+1)] 
    

    for i in range(1,n+1):
        for j in range(1, m+1):
            if str1[i-1] == str2[j-1]:
                s[i][j] = max(s[i-1][j-1]+1, s[i-1][j]-1, s[i][j-1]-1)
                if s[i][j] == s[i-1][j-1]+1:
                    backtrack[i][j] = 1
                elif s[i][j] == s[i-1][j]-1:
                    backtrack[i][j] = 2
                elif s[i][j] == s[i][j-1]-1:
                    backtrack[i][j] = 3
            else:
                s[i][j] = max(s[i-1][j-1]-1, s[i-1][j]-1, s[i][j-1]-1)
                if s[i][j] == s[i-1][j-1]-1:
                    backtrack[i][j] = 1
                elif s[i][j] == s[i-1][j]-1:
                    backtrack[i][j] = 2
                elif s[i][j] == s[i][j-1]-1:
                    backtrack[i][j] = 3                
    
    align1, align2 = '', ''
    j = m
    max_val = -1
    for x in range(m, n):
        if s[x][j] > max_val:
            max_val = s[x][j]
            i = x
    
    while (i > 0 ) and (j > 0):
        if backtrack[i][j] == 1:
            align1 += str1[i-1]
            align2 += str2[j-1]
            i = i-1
            j = j-1
        elif backtrack[i][j] == 2:
            align1 += str1[i-1]
            align2 += '-'
            i = i-1
        elif backtrack[i][j] == 3:
            align1 += '-'
            align2 += str2[j-1]
            j = j-1
    print max_val
    print align1[::-1]
    print align2[::-1]
    return 

In [56]:
str1 = 'GTAGGCTTAAGGTTA'
str2 = 'TAGATA'

In [57]:
FittingAlignment(str1, str2)

2
TAGGCTTA
TA-G-ATA


### BA5I | Overlap Alignment 

In [58]:
def OverlapAlignment(str1, str2):
    n = len(str1)
    m = len(str2)
    s = [[0 for x in range(m+1)] for y in range(n+1)]
    backtrack = [[0 for x in range(m+1)] for y in range(n+1)] 
    max_loc = (0,0)
    max_score = -float('inf')

    for i in range(1,n+1):
        for j in range(1, m+1):
            if str1[i-1] == str2[j-1]:
                s[i][j] = max(s[i-1][j-1]+1, s[i-1][j]-2, s[i][j-1]-2)
                if s[i][j] == s[i-1][j-1]+1:
                    backtrack[i][j] = 1
                elif s[i][j] == s[i-1][j]-2:
                    backtrack[i][j] = 2
                elif s[i][j] == s[i][j-1]-2:
                    backtrack[i][j] = 3
            else:
                s[i][j] = max(s[i-1][j-1]-2, s[i-1][j]-2, s[i][j-1]-2)
                if s[i][j] == s[i-1][j-1]-2:
                    backtrack[i][j] = 1
                elif s[i][j] == s[i-1][j]-2:
                    backtrack[i][j] = 2
                elif s[i][j] == s[i][j-1]-2:
                    backtrack[i][j] = 3                
    for i in range(1, n+1):
        for j in range(1, m+1):
            if s[i][m]> max_score:
                max_score = s[i][m]
                max_loc = (i, m)
            if s[n][j] > max_score:
                max_score = s[n][j]
                max_loc = (n, j)
                
    align1, align2 = '', ''
    i = max_loc[0]
    j = max_loc[1]
    
    while (i > 0 ) and (j > 0):
        if backtrack[i][j] == 1:
            align1 += str1[i-1]
            align2 += str2[j-1]
            i = i-1
            j = j-1
        elif backtrack[i][j] == 2:
            align1 += str1[i-1]
            align2 += '-'
            i = i-1
        elif backtrack[i][j] == 3:
            align1 += '-'
            align2 += str2[j-1]
            j = j-1
    print max_score
    print align1[::-1]
    print align2[::-1]
    return 

In [59]:
str1 = 'PAWHEAE'
str2 = 'HEAGAWGHEE'

In [60]:
OverlapAlignment(str1, str2)

1
HEAE
HEA-


### BA5N | Topological Ordering

In [65]:
def TopologicalOrder(graph):
    in_degree = {node:0 for node in graph}
    for u in graph:
        for v in graph[u]:
            if v not in in_degree:
                in_degree[v] = 1
            else:
                in_degree[v]+= 1

    order = []
    candidates = []
    for node in in_degree:
        if in_degree[node] == 0:
            candidates.append(node)

    while len(candidates) > 0:
        a = candidates.pop()
        order.append(a)

        if a in graph:
            for b in graph[a]:
                in_degree[b] -= 1
                if in_degree[b] == 0:
                    candidates.append(b)
    print ', '.join(node for node in order)
    return order

In [66]:
with open('../data/rosalind_ba5n.txt') as f:
    graph ={}
    lines = [line.strip().split(' -> ') for line in f.readlines()]
    for line in lines:
        from_node = line[0]
        to_node = line[1].split(',')
        if from_node not in graph:
            graph[from_node] = to_node

In [67]:
TopologicalOrder(graph)

4, 5, 1, 2, 3


['4', '5', '1', '2', '3']

### BA5M | Multiple Longest Common Subsequence Problem

In [68]:
with open('../data/rosalind_ba5m.txt') as f:
    str1, str2, str3 = (string.strip() for string in f.readlines())

In [76]:
def MultipleAlignment(str1, str2, str3):
    l, m, n = len(str1), len(str2), len(str3)
    s = [[[0 for x in range(n+1)] for y in range(m+1)] for z in range(l+1)]
    backtrack = [[[0 for x in range(n+1)] for y in range(m+1)] for z in range(l+1)]

    for i in range(1, l+1):
        for j in range(1, m+1):
            for k in range(1, n+1):
                reward = 0
                if str1[i-1] == str2[j-1] == str3[k-1]:
                    reward = 1
                s[i][j][k] = max(s[i-1][j-1][k-1]+reward,
                                 s[i][j-1][k-1],
                                 s[i-1][j][k-1],
                                 s[i-1][j-1][k],
                                 s[i][j][k-1],
                                 s[i][j-1][k],
                                 s[i-1][j][k])
                if s[i][j][k] == s[i-1][j-1][k-1]+reward:
                    backtrack[i][j][k] = 0
                elif s[i][j][k] == s[i][j-1][k-1]:
                    backtrack[i][j][k] = 1
                elif s[i][j][k] == s[i-1][j][k-1]:
                    backtrack[i][j][k] = 2
                elif s[i][j][k] == s[i-1][j-1][k]:
                    backtrack[i][j][k] = 3
                elif s[i][j][k] == s[i][j][k-1]:
                    backtrack[i][j][k] = 4
                elif s[i][j][k] == s[i][j-1][k]:
                    backtrack[i][j][k] = 5
                elif s[i][j][k] == s[i-1][j][k]:
                    backtrack[i][j][k] = 6


    align1, align2, align3 = '','',''
    i, j, k = l, m, n

    while s[i][j][k] > 0:
        if backtrack[i][j][k] == 0:
            align1 += str1[i-1]
            align2 += str2[j-1]
            align3 += str3[k-1]
            i -= 1
            j -= 1
            k -= 1
        elif backtrack[i][j][k] == 1:
            align1 += '-'
            align2 += str2[j-1]
            align3 += str3[k-1]
            j -= 1
            k -= 1
        elif backtrack[i][j][k] == 2:
            align1 += str1[i-1]
            align2 += '-'
            align3 += str3[k-1]
            i -= 1
            k -= 1
        elif backtrack[i][j][k] == 3:
            align1 += str1[i-1]
            align2 += str2[j-1]
            align3 += '-'
            i -= 1
            j -= 1
        elif backtrack[i][j][k] == 4:
            align1 += '-'
            align2 += '-'
            align3 += str3[k-1]
            k -= 1
        elif backtrack[i][j][k] == 5:
            align1 += '-'
            align2 += str2[j-1]
            align3 += '-'
            j -= 1
        elif backtrack[i][j][k] == 6:
            align1 += str1[i-1]
            align2 += '-'
            align3 += '-'
            i -= 1

    strings = [str1, str2, str3]
    alignments = [align1[::-1], align2[::-1], align3[::-1]]
    patch = ['','','']
    for i, a in enumerate(alignments):
        if a[0] != strings[i][0]:
            for j in range(len(strings[i])):
                if strings[i][j] != a[0]:
                    patch[i] += strings[i][j]
                else:
                    break
    max_l = max([len(p) for p in patch])
    patch = ['-'*(max_l - len(p))+p for p in patch]    
    for i in range(3):
        alignments[i] = patch[i] + alignments[i]

    print s[l][m][n]
    for a in alignments:
        print a

In [78]:
MultipleAlignment(str1, str2, str3)

3
AT-ATCCG-
-T---CCGA
ATGTACTG-


### BA5K | Middle Edge in Linear Space Problem

In [88]:
from Bio.SubsMat.MatrixInfo import blosum62
blosum62_update = {}
for pair in blosum62:
    blosum62_update[pair] = blosum62[pair]
    blosum62_update[(pair[1], pair[0])] = blosum62[pair]

In [89]:
score_mat = blosum62_update
penalty = 5

In [90]:
with open('../data/rosalind_ba5k.txt') as f:
    str1, str2 = (string.strip() for string in f.readlines())

In [91]:
def GlobalScore(str1, str2, penalty, score_mat):
    n = len(str1)
    m = len(str2)
    s = [[0 for x in range(m+1)] for y in range(n+1)]
    
    for i in range(n+1):
        s[i][0] = -i*penalty
    for j in range(m+1):
        s[0][j] = -j*penalty
    for i in range(1,n+1):
        for j in range(1, m+1):
            s[i][j] = max(s[i-1][j-1]+score_mat[(str1[i-1], str2[j-1])], s[i-1][j]-penalty, s[i][j-1]-penalty)
    return [s[x][m] for x in range(n+1)]

In [92]:
mid = len(str2)//2
f_col = GlobalScore(str1, str2[:mid], penalty, score_mat)
b_col = GlobalScore(str1[::-1], str2[mid-1:][::-1], penalty, score_mat)[::-1]
sum_col = map(lambda x,y:x+y, f_col, b_col)

max_val = max(sum_col)
index = sum_col.index(max_val)
#mid_node = (max(enumerate(sum_col), key=lambda x: x[1])[0], mid)

In [93]:
# s is not the same as global score 
def LinearScore(str1, str2, penalty, score_mat):
    s = [[-penalty*y, 0] for y in range(len(str1)+1)]
    s[0][1] = -penalty
    for j in range(1, len(str2)+1):
        for i in range(1, len(str1)+1):
            j1 = (j-1)%2
            j2 = j%2
            s[0][j2] = -penalty*j
            s[i][j2] = max(s[i-1][j1]+score_mat[(str1[i-1], str2[j-1])], s[i-1][j2]-penalty, s[i][j1]-penalty)

    return s, len(str2)%2

In [94]:
def MiddleEdge(str1, str2, penalty, score_mat):
    mid = len(str2)//2
    f, idx_f = LinearScore(str1, str2[:mid], 5, score_mat)
    b, idx_b = LinearScore(str1[::-1], str2[mid-1:][::-1], 5, score_mat)
    b = b[::-1]
    sum_col = map(lambda x,y:x+y, [col[idx_f] for col in f], [col[idx_b] for col in b])
    
    mid_node = (max(enumerate(sum_col), key=lambda x: x[1])[0], mid)
    
    #down
    if b[mid_node[0]+1][idx_b] == b[mid_node[0]][idx_b] + penalty:
        mid_node_to = (mid_node[0]+1, mid_node[1])
    # right
    elif b[mid_node[0]][idx_b^1] == b[mid_node[0]][idx_b] + penalty:
        mid_node_to = (mid_node[0], mid_node[1]+1)
    else: #diagnal
        mid_node_to = (mid_node[0]+1, mid_node[1]+1)
    print mid_node, mid_node_to 


In [95]:
MiddleEdge(str1, str2, 5, blosum62_update)

(4, 3) (5, 4)


### BA5J | Alignment with Affine Gap Penalties Problem

In [99]:
with open('../data/rosalind_ba5j.txt') as f:
    str1, str2 = (string.strip() for string in f.readlines())

In [100]:
def AffineGapAlignment(str1, str2, score_mat):
    n = len(str1)
    m = len(str2)
    s_lower = [[0 for x in range(m+1)] for y in range(n+1)]
    s_middle = [[0 for x in range(m+1)] for y in range(n+1)]
    s_upper = [[0 for x in range(m+1)] for y in range(n+1)]

    b_lower = [[0 for x in range(m+1)] for y in range(n+1)]
    b_middle = [[0 for x in range(m+1)] for y in range(n+1)]
    b_upper = [[0 for x in range(m+1)] for y in range(n+1)]

    open_penalty = -11
    ext_penalty = -1

    #init
    for i in range(n+1):
        s_lower[i][0] = open_penalty +(i-1)*ext_penalty
        s_middle[i][0] = open_penalty + (i-1)*ext_penalty
        s_upper[i][0] = -float('inf')
    for j in range(m+1):
        s_upper[0][j] = open_penalty + (j-1)*ext_penalty
        s_middle[0][j] = open_penalty + (j-1)*ext_penalty
        s_lower[0][j] = -float('inf')
    s_middle[0][0] = 0
    
    for i in range(1,n+1):
        for j in range(1, m+1):
            s_lower[i][j] = max([s_middle[i-1][j]+open_penalty, s_lower[i-1][j]+ext_penalty])
            b_lower[i][j] = [s_middle[i-1][j]+open_penalty, s_lower[i-1][j]+ext_penalty].index(s_lower[i][j])

            s_upper[i][j] = max([s_middle[i][j-1]+open_penalty, s_upper[i][j-1]+ext_penalty])
            b_upper[i][j] = [s_middle[i][j-1]+open_penalty, s_upper[i][j-1]+ext_penalty].index(s_upper[i][j])

            s_middle[i][j] = max([s_middle[i-1][j-1]+score_mat[(str1[i-1], str2[j-1])], s_lower[i][j], s_upper[i][j]])
            b_middle[i][j] = [s_middle[i-1][j-1]+score_mat[(str1[i-1], str2[j-1])], s_lower[i][j], s_upper[i][j]].index(s_middle[i][j])
            
    align1, align2 = '', ''
    i, j = n, m
    curr = 2 # max at middle matrix 
    while (i >0 ) and (j > 0):
        if curr == 0:
            if b_lower[i][j] == 0:
                curr = 2
            align1 += str1[i-1]
            align2 += '-'
            i = i-1

        elif curr == 1:
            if b_upper[i][j] == 0:
                curr = 2
            align1 += '-'
            align2 += str2[j-1]
            j = j-1

        elif curr == 2:
            if b_middle[i][j] == 0:
                align1 += str1[i-1]
                align2 += str2[j-1]
                i = i-1
                j = j-1
            elif b_middle[i][j] == 1:
                curr = 0
            elif b_middle[i][j] == 2:
                curr = 1

    print s_middle[n][m]
    print align1[::-1]
    print align2[::-1]
    
    return s_middle[n][m]

In [101]:
AffineGapAlignment(str1, str2, score_mat)

8
PRT---EINS
PRTWPSEIN-


8